In [1]:
import pandas as pd
# !pip install pandas pyarrow
import numpy as np
import sys

# !pip install chardet
import chardet
import chardet.universaldetector

# !pip install unidecode
import unidecode

# !pip install functools
from functools import lru_cache

import os
from os import remove
import gc

# !pip install wget
import wget

# !pip install py7zr
from py7zr import SevenZipFile

import glob

from io import StringIO

In [2]:
# Ajustando diretório de trabalho
pwd = os.getcwd()

In [6]:
def extract_rais(ano, tipo=None, regiao=None, uf=None):
    """
    - Faz o download dos arquivos da RAIS para o ano especificado.
    - O período da RAIS tem início em 1985.

    - Para os anos maiores ou iguais a 2018, são 2 tipos de arquivos, ESTAB e VINC, sendo o tipo VINC subdividido em 7 regiões:
        1. _CENTRO_OESTE
        2. _MG_ES_RJ
        3. _NI
        4. _NORDESTE
        5. _NORTE
        6. _SP
        7. _SUL
    
    - Para anos de 1999 até 2017:
        - Tipo ESTAB: 'ESTB' ou 'Estb' seguido do ano (e.g., ESTB2017)
        - Tipo VINC: 'UF' seguido do ano (e.g., SP2017)

    - Argumentos:
        - ano (int): Ano do arquivo.
        - tipo (str, opcional): Tipo de arquivo, pode ser 'ESTAB' ou 'VINC'. Se não fornecido, considera ambos.
        - regiao (str, opcional): Região para o tipo 'VINC'. Deve ser uma das seguintes: ['_CENTRO_OESTE', '_MG_ES_RJ', '_NI', '_NORDESTE', '_NORTE', '_SP', '_SUL']. Se não for fornecido e o tipo for 'VINC', considera todas as regiões.
        - uf (str, opcional): Sigla de um estado para os anos de 1999 até 2017. Exemplo: 'SP', 'RJ', 'MG'.
    """
    # Validação do ano
    if ano < 1985:
        raise ValueError("O ano deve ser maior ou igual a 1985.")
    
    # Tipos e regiões possíveis
    tipos_arquivos = ['ESTAB', 'VINC']
    tipos_regioes = ['_CENTRO_OESTE', '_MG_ES_RJ', '_NI', '_NORDESTE', '_NORTE', '_SP', '_SUL']
    ufs_brasil = [
        'AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 
        'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'
    ]

    # Validação de tipo
    tipos_selecionados = [tipo] if tipo else tipos_arquivos
    if tipo and tipo not in tipos_arquivos:
        raise ValueError(f"Tipo inválido: {tipo}. Use um dos seguintes: {tipos_arquivos}")

    # Gerar URLs
    urls = []
    
    # Para anos entre 1999 e 2017
    if ano <= 2017 and ano > 1999:
        for t in tipos_selecionados:
            if t == 'ESTAB':
                urls.append(f'ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}/ESTB{ano}.7z')
                urls.append(f'ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}/Estb{ano}.7z')  # Versão com "Estb"
            elif t == 'VINC':
                for estado in ufs_brasil:
                    urls.append(f'ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}/{estado}{ano}.7z')
    else:
        for t in tipos_selecionados:
            if t == 'ESTAB':
                urls.append(f'ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}/RAIS_{t}_PUB.7z')
            elif t == 'VINC':
                regioes_selecionadas = [regiao] if regiao else tipos_regioes
                for r in regioes_selecionadas:
                    urls.append(f'ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}/RAIS_{t}_PUB{r}.7z')
    
    # Exibir as URLs
    print("As seguintes URLs foram geradas:")
    for url in urls:
        print(url)

    # Confirmar com o usuário
    resposta = input("\nDeseja prosseguir com o download dessas URLs? (Sim/Não): ").strip().lower()
    if resposta not in ['sim', 's', 'y', 'yes']:
        print("Operação cancelada pelo usuário.")
        return

    
    # Download e extração dos arquivos
    for url in urls:
        try:
            print(f'\nBaixando arquivo: {url}')
            wget.download(url)
            print(f'\nDownload concluído: {url}')

            # Extração do arquivo
            nome_arquivo = url.split('/')[-1]
            with SevenZipFile(nome_arquivo, mode='r') as archive:
                archive.extractall()
                print(f'Extração concluída: {nome_arquivo}')

            # Remover o arquivo 7z após extração
            remove(nome_arquivo)
            print(f'Arquivo removido: {nome_arquivo}')

        except FileNotFoundError:
            print(f'\nArquivo não encontrado ou incompatível no diretório de download: {url}')
        except Exception as e:
            print(f'\nErro ao processar o arquivo {url}: {e}')

In [ ]:
extract_rais(2011)

As seguintes URLs foram geradas:
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/ESTB2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/Estb2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/AC2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/AL2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/AM2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/AP2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/BA2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/CE2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/DF2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/ES2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/GO2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/MA2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/MG2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/MS2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/MT2011.7z
ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/2011/PA2011.7z
ftp://ftp.mtps.gov.br/pdet/microdad